# 06 — Modules 7–8: Entry signals & Position sizing

Signals answer:
> Is today a valid entry day?

We implement:
- **Breakout**: close > highest close of prior N days
- **Pullback reclaim**: yesterday below MA, today above MA

Risk sizing answers:
> If I enter, how many shares can I buy without exceeding risk rules?

We use:
- account size (e.g. 500€)
- risk per trade (e.g. 1%)
- ATR-based stop distance

In [1]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [8]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.screeners.universe import UniverseConfig, UniverseFilterConfig, eligible_universe
from swing_screener.screeners.ranking import RankingConfig, top_candidates
from swing_screener.signals.entries import EntrySignalConfig, build_signal_board
from swing_screener.risk.position_sizing import RiskConfig, build_trade_plans

tickers = [
    "AAPL",
    "MSFT",
    "NVDA",
    "AMZN",
    "META",
    "GOOGL",
    "TSLA",
    "AMD",
    "NFLX",
    "AVGO",
    "INTC",
    "ORCL",
    "CRM",
    "QCOM",
    "ADBE",
    "CSCO",
    "SHOP",
    "UBER",
    "ABNB",
    "SPY",
]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))

univ = eligible_universe(ohlcv, UniverseConfig(
    filt=UniverseFilterConfig(min_price=10, max_price=3000, max_atr_pct=12, require_trend_ok=True)
))

ranked = top_candidates(univ, RankingConfig(top_n=5))
signals = build_signal_board(ohlcv, ranked.index.tolist(), EntrySignalConfig(breakout_lookback=50, pullback_ma=20))

ranked_small = ranked.copy()
plans = build_trade_plans(ranked_small, signals, RiskConfig(account_size=50000, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60))

signals

,last,breakout50,breakout_level,pullback_ma20,ma20_level,signal
ticker,,,,,,
GOOGL,335.970001,True,331.859985,False,315.156001,breakout
INTC,47.290001,True,45.549999,False,38.968000,breakout
AMD,220.970001,False,259.649994,True,211.988499,pullback
TSLA,447.200012,False,489.880005,False,461.338503,none
SHOP,167.440002,False,173.860001,False,165.994501,none


In [9]:
plans

,signal,entry,stop,atr14,k_atr,shares,position_value,risk_amount_target,risk_per_share,realized_risk,max_position_value
ticker,,,,,,,,,,,
INTC,breakout,47.29,42.88,2.2057,2.0,113,5343.77,500.0,4.4114,498.49,30000.0
GOOGL,breakout,335.97,322.80,6.5836,2.0,37,12430.89,500.0,13.1671,487.18,30000.0
AMD,pullback,220.97,205.69,7.6414,2.0,32,7071.04,500.0,15.2829,489.05,30000.0


If `plans` is empty with a small account, it's normal: many large caps are expensive vs your constraints.
Try increasing account size, relaxing max_position_pct, or using cheaper tickers/ETFs.